In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

In [23]:
def reduce_indices(indices_A, indices_B, partition):
    
    indices_A, indices_B = np.array(indices_A), np.array(indices_B)
    new_indices_A, new_indices_B = [], []
    
    for i in indices_A:
        new_indices_A.append( i - np.count_nonzero(partition[:i] == 'J') )
    for i in indices_B:
        new_indices_B.append( i - np.count_nonzero(partition[:i] == 'I') )
    
    return new_indices_A, new_indices_B

In [6]:
A = gqcpy.AdjacencyMatrix.Cyclic(4)

t = 1.0
U = 3.5
H = gqcpy.HoppingMatrix(A, t, U)

hubbard_hamiltonian = gqcpy.HubbardHamiltonian(H)
onv_basis = gqcpy.SpinResolvedONVBasis(4, 2, 2)

solver = gqcpy.EigenproblemSolver.Dense_d()
environment = gqcpy.CIEnvironment.Dense(hubbard_hamiltonian, onv_basis)

wfn = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
coefficients = wfn.coefficients()

coefficients

array([-5.854e-02, -1.354e-01, -1.038e-16,  1.046e-16,  1.354e-01,  2.403e-01, -1.354e-01, -1.292e-16,  1.354e-01,
        1.354e-01,  4.806e-01,  1.354e-01, -5.745e-17,  1.354e-01,  5.854e-02,  2.403e-01,  1.354e-01,  4.777e-17,
       -5.728e-17,  1.354e-01,  2.403e-01,  5.854e-02,  1.354e-01,  5.009e-17,  1.354e-01,  4.806e-01,  1.354e-01,
        1.354e-01, -1.040e-16, -1.354e-01,  2.403e-01,  1.354e-01, -3.089e-17, -4.547e-17, -1.354e-01, -5.854e-02])

In [7]:
partition = np.array(['J', 'I', 'J', 'I'])

In [8]:
def schmidt_decomposition(onv_a, I_a, onv_b, I_b):
    
    address = onv_basis.compoundAddress(I_a, I_b)
    print("Address: {}\tONV:\t".format(address), onv_a, "|", onv_b)
    
    operator_string_alpha = gqcpy.SpinUnresolvedOperatorString.FromONV(onv_a)
    operator_string_beta = gqcpy.SpinUnresolvedOperatorString.FromONV(onv_b)
    
    K_system = np.count_nonzero(partition == 'I')
    K_environment = np.count_nonzero(partition== 'J')

    alpha_partition = partition[operator_string_alpha.operatorIndices()]
    beta_partition = partition[operator_string_beta.operatorIndices()]
    
    system_alpha, environment_alpha = operator_string_alpha.schmidtDecomposition(alpha_partition)
    system_beta, environment_beta = operator_string_beta.schmidtDecomposition(beta_partition)
    
    print("The alpha system consists of sites: {} and has phase factor {}.".format(system_alpha.operatorIndices(), system_alpha.phaseFactor()))
    print("The alpha environment consists of sites: {} and has phase factor {}.".format(environment_alpha.operatorIndices(), environment_alpha.phaseFactor()))
    print("The beta system consists of sites: {} and has phase factor {}.".format(system_beta.operatorIndices(), system_beta.phaseFactor()))
    print("The beta environment consists of sites: {} and has phase factor {}.".format(environment_beta.operatorIndices(), environment_beta.phaseFactor()))
    
    system_alpha_indices, environment_alpha_indices = reduce_indices(system_alpha.operatorIndices(), environment_alpha.operatorIndices(), partition)
    system_beta_indices, environment_beta_indices = reduce_indices(system_beta.operatorIndices(), environment_beta.operatorIndices(), partition)
    
    print("Alpha system: correction of indices {} to {}.".format(system_alpha.operatorIndices(), system_alpha_indices))
    print("Beta system: correction of indices {} to {}.".format(system_beta.operatorIndices(), system_beta_indices))
    print("Alpha environment: correction of indices {} to {}.".format(environment_alpha.operatorIndices(), environment_alpha_indices))
    print("Beta environment: correction of indices {} to {}.".format(environment_beta.operatorIndices(), environment_beta_indices))
    
    onv_system_alpha = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(system_alpha_indices, K_system)
    onv_system_beta = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(system_beta_indices, K_system)
    onv_system = gqcpy.SpinResolvedONV(onv_system_alpha, onv_system_beta)
    print("The spin-resolved ONV of the system is: {}.".format(onv_system))
    # Note: read ONVs from right to left due to bitstring representation.
    
    coefficients[address] *= system_alpha.phaseFactor() * system_beta.phaseFactor()

    onv_environment_alpha = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(environment_alpha_indices, K_environment)
    onv_environment_beta = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(environment_beta_indices, K_environment)
    onv_environment = gqcpy.SpinResolvedONV(onv_environment_alpha, onv_environment_beta)
    print("The spin-resolved ONV of the environment is: {}.\n".format(onv_environment))
    
    onvs.append((onv_system, onv_environment))
    # Note: read ONVs from right to left due to bitstring representation.

In [9]:
onvs = []
onv_basis.forEach(schmidt_decomposition)

Address: 0	ONV:	 0011 | 0011
The alpha system consists of sites: [1] and has phase factor -1.
The alpha environment consists of sites: [0] and has phase factor 1.
The beta system consists of sites: [1] and has phase factor -1.
The beta environment consists of sites: [0] and has phase factor 1.
Alpha system: correction of indices [1] to [0].
Beta system: correction of indices [1] to [0].
Alpha environment: correction of indices [0] to [0].
Beta environment: correction of indices [0] to [0].
The spin-resolved ONV of the system is: 01|01.
The spin-resolved ONV of the environment is: 01|01.

Address: 1	ONV:	 0011 | 0101
The alpha system consists of sites: [1] and has phase factor -1.
The alpha environment consists of sites: [0] and has phase factor 1.
The beta system consists of sites: [] and has phase factor 1.
The beta environment consists of sites: [0, 2] and has phase factor 1.
Alpha system: correction of indices [1] to [0].
Beta system: correction of indices [] to [].
Alpha environmen

In [10]:
coefficients

array([-5.854e-02,  1.354e-01,  1.038e-16,  1.046e-16, -1.354e-01,  2.403e-01,  1.354e-01, -1.292e-16,  1.354e-01,
       -1.354e-01,  4.806e-01, -1.354e-01,  5.745e-17,  1.354e-01,  5.854e-02, -2.403e-01,  1.354e-01, -4.777e-17,
       -5.728e-17, -1.354e-01, -2.403e-01,  5.854e-02, -1.354e-01,  5.009e-17, -1.354e-01,  4.806e-01,  1.354e-01,
       -1.354e-01, -1.040e-16,  1.354e-01,  2.403e-01, -1.354e-01,  3.089e-17, -4.547e-17,  1.354e-01, -5.854e-02])

In [18]:
K_system = np.count_nonzero(partition == 'I')
K_environment = np.count_nonzero(partition== 'J')
N_system = 1

onv_alpha_system = gqcpy.SpinUnresolvedONV(K_system, N_system, 1)
onv_beta_system = gqcpy.SpinUnresolvedONV(K_system, N_system, 2)
onv_system = gqcpy.SpinResolvedONV(onv_alpha_system, onv_beta_system)

01|10

In [21]:
for i,onv_pair in enumerate(onvs):
    if onv_pair[0] == onv_system:
        print("joepie: ", i)

joepie:  3
joepie:  5
joepie:  15
joepie:  17


In [22]:
onvs

[(01|01, 01|01),
 (01|00, 01|11),
 (01|01, 01|10),
 (01|10, 01|01),
 (01|11, 01|00),
 (01|10, 01|10),
 (00|01, 11|01),
 (00|00, 11|11),
 (00|01, 11|10),
 (00|10, 11|01),
 (00|11, 11|00),
 (00|10, 11|10),
 (01|01, 10|01),
 (01|00, 10|11),
 (01|01, 10|10),
 (01|10, 10|01),
 (01|11, 10|00),
 (01|10, 10|10),
 (10|01, 01|01),
 (10|00, 01|11),
 (10|01, 01|10),
 (10|10, 01|01),
 (10|11, 01|00),
 (10|10, 01|10),
 (11|01, 00|01),
 (11|00, 00|11),
 (11|01, 00|10),
 (11|10, 00|01),
 (11|11, 00|00),
 (11|10, 00|10),
 (10|01, 10|01),
 (10|00, 10|11),
 (10|01, 10|10),
 (10|10, 10|01),
 (10|11, 10|00),
 (10|10, 10|10)]